    # metacognitive prompting

### import statements

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
from dotenv import load_dotenv
import sys

load_dotenv()

In [ ]:
sys.path.append(os.getenv("SYS_PATH"))
print(f'{os.getenv("SYS_PATH")}\n\n')
print(sys.path)

In [ ]:
from openai import OpenAI
from ipywidgets import widgets, Output
from IPython.display import display
import threading

from models.user import create_user
from models.chat_message import create_chat_message
from models.conversation import create_conversation
from models.chatbot_response import create_chatbot_response
from models.expectation import create_expectation
from models.expectation_revision import create_expectation_revision
from models.violation import create_violation

from utils.model_operations import create_model, get_model, update_model, delete_model
from bson.objectid import ObjectId
from typing import Optional

In [ ]:
%run ../services/mongo_service.py

In [ ]:
from services.mongo_service import mongo_client, ping_client, close_client
db_client = mongo_client()
ping_client(db_client)

In [ ]:
text_input = ""
set_user = None
set_conversation = None
chat_output=Output()
active = True
processing_message = threading.Event()

In [ ]:
test_user = create_user(email="adelbert.baldemar@gmail.com", password="supersecret")
print(f"test_user: {test_user}\nis of type{type(test_user)}")
saved_user_id = create_model(collection_name="users", model_data=test_user, client=db_client)

In [ ]:
test_conversation = create_conversation(user_id=saved_user_id)
saved_conversation_id = create_model(collection_name="conversations", model_data=test_conversation, client=db_client)

In [ ]:
test_expectation = create_expectation(reasoning="user is feeling nervous", user_predictions=["Hi Chat", "How do I build confidence?", "Why does no one like me?", "I always feel nervous, help me find a way to calm down."], additional_data=["23 years old", "male", "full name is Adelbert Baldemar", "highly neurotic personality", "passionate about music, mainly contemporary classical composers such as Olivier Messiaen and Alfred Schnittke"])
saved_expectation_id = create_model(collection_name="expectations", model_data=test_expectation, client=db_client)

In [ ]:
test_expectation_revision = create_expectation_revision(revised_input_possibilities=["Hi Chat, my name is Adelbert Baldemar. I'm a 23-year-old male who's passionate about music, particularly contemporary classical composers like Olivier Messiaen and Alfred Schnittke.", "How can I build confidence, considering I'm highly neurotic?", "Why do I feel like no one likes me, especially considering my neurotic personality? Is it related to my passion for contemporary classical composers like Olivier Messiaen and Alfred Schnittke?", "I often feel nervous, especially in social situations. Can you suggest ways, considering my love for music, to help me calm down? Maybe something related to Messiaen or Schnittke?"], prediction_error=-0.045, initial_expectation_id=saved_expectation_id)
saved_expectation_revision_id = create_model(collection_name="expectation_revisions", model_data=test_expectation_revision, client=db_client)

In [ ]:
test_message = create_chat_message(user_id=saved_user_id, content="Hi there! I've been exploring new hobbies lately, and I stumbled upon medieval choir music. It's intriguing, and I like the timbre and atmosphere of it, but the harmony behind it is often rather simple. Do you have any recommendations for pieces or composers to listen to that I might find more interesting?", conversation_id=saved_conversation_id)
saved_message_id = create_model(collection_name="chat_messages", model_data=test_message, client=db_client)

In [ ]:
test_violation = create_violation(last_llm_response_id=None, expectation_id=saved_expectation_id, voe_thought="The user's actual input reflects a curiosity about medieval choir music, which seems unrelated to the predictions generated based on the user's personality traits and interests. While the predictions anticipated inquiries about confidence-building strategies, feelings of being disliked, and methods to calm down in social situations related to contemporary classical composers, the user's input diverges into a different realm of musical exploration. This discrepancy suggests that the user's interests and inquiries may extend beyond the scope of contemporary classical composers like Olivier Messiaen and Alfred Schnittke, as the user's engagement with medieval choir music demonstrates. It highlights the multifaceted nature of human interests and preferences, indicating that individuals may have diverse tastes and curiosities that cannot always be predicted based solely on certain personality traits or hobbies.")
saved_violation_id = create_model(collection_name="violations", model_data=test_violation, client=db_client)

In [ ]:
get_model(collection_name="users", model_id=saved_user_id, client=db_client)

In [ ]:
def initialize_chat(user_id: ObjectId, conversation_id: Optional[ObjectId] = None):
    global set_user, set_conversation
    set_user = get_model(collection_name="users", model_id=saved_user_id, client=db_client)
    if conversation_id:
        set_conversation = get_model(collection_name="conversations", model_id=conversation_id, client=db_client)
    else:
        new_conversation = create_conversation(user_id=user_id)
        conversation_id = create_model(collection_name="conversations", model_data=new_conversation, client=db_client)
    set_conversation = get_model(collection_name="conversations", model_id=conversation_id, client=db_client)

In [ ]:
apikey = os.getenv("OPENAI_API_KEY")
openai_client = OpenAI(
    api_key=apikey
)
print(openai_client)

:::{admonition} `stream_chatbot` docs
:class: dropdown 

### `stream_chatbot(message)`

This function interacts with the chatbot using stream functionality.

**Parameters:**
- `message`: The message sent by the user to the chatbot.

**Returns:**
- None

**Functionality:**
- Sends the user message to the chatbot.
- Receives and prints the response from the chatbot.
```

In [ ]:
def stream_chatbot(message):
    global chat_output
    chat_completion_stream = openai_client.chat.completions.create(
        messages=[
            {
                "role":"user",
                "content":message,
             },
        ],
        model="gpt-3.5-turbo",
        stream=True
    )
    print(chat_completion_stream)
    for chunk in chat_completion_stream:
        if chunk.choices[0].delta.content is not None:
            chat_output.append_stdout(f"{chunk.choices[0].delta.content}")
    chat_output.append_stdout(f"\n")

:::{admonition} `chatbot` docs
:class: dropdown

### `chatbot(user_message)`

This function interacts with the chatbot based on user input.

**Parameters:**
- `user_message`: The message provided by the user.

**Returns:**
- None

**Functionality:**
- Initiates interaction with the chatbot by passing the user's message.
- Prints the chatbots response.


In [ ]:
def chatbot(user_message):
    global active
    global chat_output
    processing_message.set()
    chat_output.append_stdout("Chatbot: ")
    if user_message.lower() in ["bye!", "quit", "exit"]:
        chat_output.append_stdout("BYE\n")
        active = False
    else:
        stream_chatbot(user_message)
    processing_message.clear()

:::{admonition} `print_user_message` docs
:class: dropdown

### `print_user_message(user_message)`

This function prints the user's message.

**Parameters:**
- `user_message`: The message provided by the user.

**Returns:**
- None

**Functionality:**
- Prints the user's message in the format: "You: [user_message]".


In [ ]:
def print_user_message(user_message):
    global chat_output
    chat_output.append_stdout(f"You: {user_message}\n")

In [ ]:
def handle_user_message(user_message):
    print_user_message(user_message)
    chatbot(user_message)

In [ ]:
def on_submit_button_clicked(b):
    if not processing_message.is_set():
        handle_user_message(text_input.value)
        text_input.value=""

In [ ]:
def chat_interface():
    global text_input
    text_input = widgets.Text(description="Your message: ")
    submit_button = widgets.Button(description="Submit", disabled=False)
    
    submit_button.on_click(on_submit_button_clicked)    
    
    display(chat_output, text_input, submit_button)

In [ ]:
def main():
    initialize_chat(saved_user_id)
    chat_interface()

if __name__ == "__main__": main()

In [ ]:
close_client(client=db_client)